# 基于MobileNetV2实现分类任务

## MobileNetV2简介

MobileNetV2在MobileNetV1的深度可分离卷积结构的基础上，又提出了具有线性瓶颈的倒置残差结构（Inverted Residual Block），该结构先将输入的低维度的表示先扩展到高维度，然后进行轻量级深度卷积运算，最后进行线性的卷积运算将高维度表示再压缩为低维度的表示。

作者之所以采用这种结构，是因为实验证明当输入的通道数很小的时候进行非线性激活会丢失很多信息。这种先扩张输入的通道数，再进行非线性激活，最后再对压缩后的输入进行线性变换的结构，可以在计算的过程中最大程度地保留有效信息，进而提高模型的分类准确率。

## 算法解析

MobileNetV2的主要思想是希望可以用最少的参数最大程度的保留输入中的有效信息。为此，本文作者提出了三种结构来减少参数量和计算过程中的信息损失，分别是深度可分离卷积，线性瓶颈（Linear Bottlenecks），倒置残差结构。

其中深度可分离卷积已在MobileNetV1中做过详细的解析，在此不再赘述。

### 线性瓶颈（Linear Bottlenecks）

原文中作者使用了“兴趣流形（manifold of interest）”来表示输入经过激活（activations）所获得有效信息的形态（如图1所示）。为了避免使用该专有名词解释线性瓶颈的原理可能带来的混淆，我们统一使用“有效信息”来代指“兴趣流形（manifold of interest）”。

![图1](./images/Manifold_of_Interest.png)
<center><i>图1</i></center>

以ReLU激活函数（$F(x) = max(0, x)$）为例，如图1所示，使用ReLU激活时，输入的通道数越小，丢失的信息越多；反之，输入的通道越大，丢失的信息越少。

举例来说，当通道数为2时，信息都集中在这两个通道中，如果有部分数值小于0就会被RELU激活丢失掉。而当通道数为30时，信息是分散且冗余的，所以使用RELU激活后归于0的值可能并不会影响太多信息的存储。如果经过ReLU激活输出是非0的，那输入和输出之间是做了一个线性变换的。换句话来说，ReLU的作用是线性分类器。

综上所述，我们可以得到激活函数变换的两个属性：

1. 如果有效信息在ReLU变换后仍是非0，则相当于对有效信息进行了一次线性变换。
2. ReLU能够保留有关输入的完整有效信息的前提是输入的有效信息位于输入空间的低维子空间中（即输入空间的有效信息可以完整的映射到其低维子空间上）。

针对激活函数变换的这两个属性，为了最大程度的保留输入信息中的有效信息，作者建议对于通道数较少的层做线性激活。如果需要使用ReLU激活，则需要先扩展输入的通道数再做ReLU激活。基于此，作者提出了线性瓶颈结构，如图2所示。

![图2](./images/Evolution_of_Separable_Convolution_Blocks.png)
<center><i>图2</i></center>

作者通过图2展示了可分离卷积的演化过程。（a）是标准卷积；（b）是可分离卷积；（d）和（c）都是带有线性瓶颈的可分离卷积，（d）是（c）的下一个连接状态，同样是将标准卷积拆分为深度卷积和逐点卷积，再对压缩后的输入不再使用非线性激活，而是使用线性变换，防止非线性破坏过多的信息。

### 倒置残差（Inverted Residual）

基于我们之前对线性瓶颈的分析可知，通道数比较少的输入有效信息特别集中，而使用非线性激活会损失较多的有效信息。因此，作者想到对通道数比较少的输入进行通道数的扩张。这一点不同于传统的残差模块，传统的残差模块是先进行通道数的缩减，然后扩张通道数，由之前的分析可知，这样的结构在对通道数缩减后的输入进行非线性激活时会损失大量的有效信息。

作者通过对传统的残差模块进行倒置，从而减少了模型的有效信息的损失。同时，使用捷径（shortcut）结构还是可以发挥和传统的残差模型一样的作用，提高模型中模块之间梯度传播的能力。传统残差模块和倒置残差模块的结构对比如图3所示。

![图3](./images/Inverted_Residual_Block.png)
<center><i>图3</i></center>

## 模型结构

下面我们通过MindSpore vision套件来剖析MobileNetV2的结构，相关模块在Vision套件中都有API可直接调用。

### ConvNormActivation结构

ConvNormActivation模块是所有卷积网络中最基础的模块，由一个卷积层（Conv, Depwise Conv），一个归一化层(BN)，一个激活函数组成。图2中可以套用这个结构的的小模块：Depwise Conv+BN+ReLU6，Pointwise Conv+BN+ReLU6。

In [ ]:
from typing import Optional, List

from mindspore import nn


class ConvNormActivation(nn.Cell):
    """
    Convolution/Depthwise fused with normalization and activation
    blocks definition.
    """
    def __init__(self,
                 in_planes: int,
                 out_planes: int,
                 kernel_size: int = 3,
                 stride: int = 1,
                 groups: int = 1,
                 norm: Optional[nn.Cell] = nn.BatchNorm2d,
                 activation: Optional[nn.Cell] = nn.ReLU
                 ) -> None:
        super(ConvNormActivation, self).__init__()
        padding = (kernel_size - 1) // 2
        # 设置和添加卷积层
        layers = [
            nn.Conv2d(
                in_planes,
                out_planes,
                kernel_size,
                stride,
                pad_mode='pad',
                padding=padding,
                group=groups
            )
        ]
        # 判断是否设置归一化层
        if norm:
            # 设置归一化层
            layers.append(norm(out_planes))
        # 判断是否设置激活函数
        if activation:
            # 设置激活函数
            layers.append(activation())

        self.features = nn.SequentialCell(layers)

    def construct(self, x):
        output = self.features(x)
        return output

### 倒置残差结构（Inverted Residual Block）

MobileNetv2模型的基础结构是Inverted Residual Block，其结构图如图3（b）所示。

In [ ]:
from mindspore.ops.operations import Add

from mindvision.classification.models.blocks import ConvNormActivation


class InvertedResidual(nn.Cell):
    """
    Mobilenetv2 residual block definition.
    """

    def __init__(self,
                 in_channel: int,
                 out_channel: int,
                 stride: int,
                 expand_ratio: int,
                 norm: Optional[nn.Cell] = None,
                 last_relu: bool = False
                 ) -> None:
        super(InvertedResidual, self).__init__()
        assert stride in [1, 2]

        if not norm:
            norm = nn.BatchNorm2d

        hidden_dim = round(in_channel * expand_ratio)
        self.use_res_connect = stride == 1 and in_channel == out_channel

        layers: List[nn.Cell] = []
        if expand_ratio != 1:
            # pw
            layers.append(
                ConvNormActivation(in_channel, hidden_dim, kernel_size=1,
                                   norm=norm, activation=nn.ReLU6)
            )
        layers.extend([
            # dw
            ConvNormActivation(
                hidden_dim,
                hidden_dim,
                stride=stride,
                groups=hidden_dim,
                norm=norm,
                activation=nn.ReLU6
            ),
            # pw-linear
            nn.Conv2d(hidden_dim, out_channel, kernel_size=1,
                      stride=1, has_bias=False),
            norm(out_channel)
        ])
        self.conv = nn.SequentialCell(layers)
        self.add = Add()
        self.last_relu = last_relu
        self.relu = nn.ReLU6()

    def construct(self, x):
        identity = x
        x = self.conv(x)
        if self.use_res_connect:
            x = self.add(identity, x)
        if self.last_relu:
            x = self.relu(x)
        return x

### 基准模型结构

MobileNetV2的主体结构的各项参数如图4所示。

![图4](./images/MobileNetV2_Structure.png)
<center><i>图4</i></center>

根据图4的参数，我们构造了MobileNetV2的主体结构，如下面的代码所示。

In [ ]:
from mindvision.classification.models.utils import make_divisible


class MobileNetV2(nn.Cell):
    """
    MobileNetV2 architecture.
    """

    def __init__(self,
                 alpha: float = 1.0,
                 inverted_residual_setting: Optional[List[List[int]]] = None,
                 round_nearest: int = 8,
                 block: Optional[nn.Cell] = None,
                 norm: Optional[nn.Cell] = None,
                 ) -> None:
        super(MobileNetV2, self).__init__()

        if not block:
            block = InvertedResidual
        if not norm:
            norm = nn.BatchNorm2d

        input_channel = make_divisible(32 * alpha, round_nearest)
        last_channel = make_divisible(1280 * max(1.0, alpha), round_nearest)

        # Setting of inverted residual blocks.
        if not inverted_residual_setting:
            inverted_residual_setting = [
                # t, c, n, s
                [1, 16, 1, 1],
                [6, 24, 2, 2],
                [6, 32, 3, 2],
                [6, 64, 4, 2],
                [6, 96, 3, 1],
                [6, 160, 3, 2],
                [6, 320, 1, 1],
            ]

        # Building first layer.
        features: List[nn.Cell] = [
            ConvNormActivation(3, input_channel, stride=2, norm=norm, activation=nn.ReLU6)
        ]

        # Building inverted residual blocks.
        # t: The expansion factor.
        # c: Number of output channel.
        # n: Number of block.
        # s: First block stride.
        for t, c, n, s in inverted_residual_setting:
            output_channel = make_divisible(c * alpha, round_nearest)
            for i in range(n):
                stride = s if i == 0 else 1
                features.append(block(input_channel, output_channel, stride,
                                      expand_ratio=t, norm=norm))
                input_channel = output_channel

        # Building last several layers.
        features.append(
            ConvNormActivation(input_channel, last_channel, kernel_size=1,
                               norm=norm, activation=nn.ReLU6)
        )
        # Make it nn.CellList.
        self.features = nn.SequentialCell(features)

    def construct(self, x):
        x = self.features(x)
        return x

### MobileNetV2模型

基于MobileNetV2的主体结构，我们构造了MobileNetV2分类模型，代码如下所示。

In [ ]:
from mindvision.classification.models.classifiers import BaseClassifier
from mindvision.classification.models.head import ConvHead
from mindvision.classification.models.neck import GlobalAvgPooling
from mindvision.classification.utils.model_urls import model_urls
from mindvision.utils.load_pretrained_model import LoadPretrainedModel


def mobilenet_v2(num_classes: int = 1001,
                 alpha: float = 1.0,
                 round_nearest: int = 8,
                 pretrained: bool = False,
                 resize: int = 224,
                 block: Optional[nn.Cell] = None,
                 norm: Optional[nn.Cell] = None,
                 ) -> MobileNetV2:
    """
    Constructs a MobileNetV2 architecture from
    `MobileNetV2: Inverted Residuals and Linear Bottlenecks
    <https://arxiv.org/abs/1801.04381>`_.
    """
    backbone = MobileNetV2(alpha=alpha, round_nearest=round_nearest, block=block, norm=norm)
    neck = GlobalAvgPooling(keep_dims=True)
    inp_channel = make_divisible(1280 * max(1.0, alpha), round_nearest)
    head = ConvHead(input_channel=inp_channel, num_classes=num_classes)
    model = BaseClassifier(backbone, neck, head)

    if pretrained:
        # Download the pre-trained checkpoint file from url, and load
        # checkpoint file.
        arch = "mobilenet_v2_" + str(alpha) + "_" + str(resize)
        LoadPretrainedModel(model, model_urls[arch]).run()

    return model

## 模型训练与推理

本案例基于MindSpore-GPU版本，在单GPU卡上完成模型训练和验证。

首先导入相关模块，配置相关超参数并读取数据集，该部分代码在Vision套件中都有API可直接调用，详情可以参考以下链接：https://gitee.com/mindspore/vision 。

可通过:http://image-net.org/ 进行数据集下载。

加载前先定义数据集路径，请确保你的数据集路径如以下结构。

```text
.ImageNet/
    ├── ILSVRC2012_devkit_t12.tar.gz
    ├── train/
    ├── val/
    └── mobilenetv2_infer.png
```

### 模型训练

训练模型前，需要先按照论文中给出的参数设置损失函数，优化器以及回调函数，MindSpore Vision套件提供了提供了相应的接口，具体代码如下所示。

In [ ]:
import argparse

from mindspore import context
from mindspore.common import set_seed
from mindspore.communication import init, get_rank, get_group_size
from mindspore.context import ParallelMode
from mindspore.train import Model
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig

from mindvision.classification.dataset import ImageNet
from mindvision.engine.loss import CrossEntropySmooth
from mindvision.engine.callback import LossMonitor

set_seed(1)


def mobilenet_v2_train(args_opt):
    """MobileNetV2 train."""
    context.set_context(mode=context.GRAPH_MODE, device_target=args_opt.device_target)

    # Data Pipeline.
    if args_opt.run_distribute:
        init("nccl")
        rank_id = get_rank()
        device_num = get_group_size()
        context.set_auto_parallel_context(device_num=device_num,
                                          parallel_mode=ParallelMode.DATA_PARALLEL,
                                          gradients_mean=True)
        dataset = ImageNet(args_opt.data_url,
                           split="train",
                           num_parallel_workers=args_opt.num_parallel_workers,
                           shuffle=True,
                           resize=args_opt.resize,
                           num_shards=device_num,
                           shard_id=rank_id,
                           batch_size=args_opt.batch_size,
                           repeat_num=args_opt.repeat_num)
        ckpt_save_dir = args_opt.ckpt_save_dir + "_ckpt_" + str(rank_id) + "/"
    else:
        dataset = ImageNet(args_opt.data_url,
                           split="train",
                           num_parallel_workers=args_opt.num_parallel_workers,
                           shuffle=True,
                           resize=args_opt.resize,
                           batch_size=args_opt.batch_size,
                           repeat_num=args_opt.repeat_num)
        ckpt_save_dir = args_opt.ckpt_save_dir

    dataset_train = dataset.run()
    step_size = dataset_train.get_dataset_size()

    # Create model.
    network = mobilenet_v2(args_opt.num_classes, alpha=args_opt.alpha, pretrained=args_opt.pretrained,
                           resize=args_opt.resize)

    # Set lr scheduler.
    if args_opt.lr_decay_mode == 'cosine_decay_lr':
        lr = nn.cosine_decay_lr(min_lr=args_opt.min_lr, max_lr=args_opt.max_lr,
                                total_step=args_opt.epoch_size * step_size, step_per_epoch=step_size,
                                decay_epoch=args_opt.decay_epoch)
    elif args_opt.lr_decay_mode == 'piecewise_constant_lr':
        lr = nn.piecewise_constant_lr(args_opt.milestone, args_opt.learning_rates)

    # Define optimizer.
    network_opt = nn.Momentum(network.trainable_params(), lr, args_opt.momentum)

    # Define loss function.
    network_loss = CrossEntropySmooth(sparse=True, reduction="mean", smooth_factor=args_opt.smooth_factor,
                                      classes_num=args_opt.num_classes)
    # Define metrics.
    metrics = {'acc'}

    # Set the checkpoint config for the network.
    ckpt_config = CheckpointConfig(
        save_checkpoint_steps=step_size,
        keep_checkpoint_max=args_opt.keep_checkpoint_max)
    ckpt_callback = ModelCheckpoint(prefix='mobilenet_v2',
                                    directory=ckpt_save_dir,
                                    config=ckpt_config)
    # Init the model.
    model = Model(network, loss_fn=network_loss, optimizer=network_opt, metrics=metrics)

    # Begin to train.
    model.train(args_opt.epoch_size,
                dataset_train,
                callbacks=[ckpt_callback, LossMonitor(lr)],
                dataset_sink_mode=args_opt.dataset_sink_mode)


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='MobileNetV2 train.')
    parser.add_argument('--device_target', type=str, default="GPU", choices=["Ascend", "GPU", "CPU"])
    parser.add_argument('--data_url', required=True, default=None, help='Location of data.')
    parser.add_argument('--epoch_size', type=int, default=200, help='Train epoch size.')
    parser.add_argument('--pretrained', type=bool, default=False, help='Load pretrained model.')
    parser.add_argument('--keep_checkpoint_max', type=int, default=10, help='Max number of checkpoint files.')
    parser.add_argument('--ckpt_save_dir', type=str, default="./mobilenet_v2", help='Location of training outputs.')
    parser.add_argument('--num_parallel_workers', type=int, default=8, help='Number of parallel workers.')
    parser.add_argument('--batch_size', type=int, default=64, help='Number of batch size.')
    parser.add_argument('--repeat_num', type=int, default=1, help='Number of repeat.')
    parser.add_argument('--num_classes', type=int, default=1001, help='Number of classification.')
    parser.add_argument('--lr_decay_mode', type=str, default="cosine_decay_lr", help='Learning rate decay mode.')
    parser.add_argument('--min_lr', type=float, default=0.0, help='The minimum learning rate.')
    parser.add_argument('--max_lr', type=float, default=0.1, help='The maximum learning rate.')
    parser.add_argument('--decay_epoch', type=int, default=200, help='Number of decay epochs.')
    parser.add_argument('--milestone', type=list, default=None, help='A list of milestone.')
    parser.add_argument('--learning_rates', type=list, default=None, help='A list of learning rates.')
    parser.add_argument('--momentum', type=float, default=0.9, help='Momentum for the moving average.')
    parser.add_argument('--smooth_factor', type=float, default=0.1, help='Label smoothing factor.')
    parser.add_argument('--dataset_sink_mode', type=bool, default=True, help='The dataset sink mode.')
    parser.add_argument('--run_distribute', type=bool, default=True, help='Distributed parallel training.')
    parser.add_argument('--alpha', type=float, default=1.0, help='Magnification factor.')
    parser.add_argument('--resize', type=int, default=224, help='Resize the height and weight of picture.')

    args = parser.parse_known_args()[0]
    mobilenet_v2_train(args)

运行如下脚本，即可开始训练MobileNetV2模型。

```shell
mpirun -n 8 python mobilenet_v2_imagenet_train.py --alpha 1.0 --resize 224 --data_url ./dataset/imagenet
```

```text
Epoch:[0/200], step:[2502/2502], loss:[4.676/4.676], time:872084.093, lr:0.10000
Epoch time:883614.453, per step time:353.163, avg loss:4.676
Epoch:[1/200], step:[2502/2502], loss:[4.452/4.452], time:693370.244, lr:0.09998
Epoch time:693374.709, per step time:277.128, avg loss:4.452
Epoch:[2/200], step:[2502/2502], loss:[3.885/3.885], time:685880.388, lr:0.09990
Epoch time:685884.401, per step time:274.134, avg loss:3.885
Epoch:[3/200], step:[2502/2502], loss:[3.550/3.550], time:689409.851, lr:0.09978
Epoch time:689413.237, per step time:275.545, avg loss:3.550
Epoch:[4/200], step:[2502/2502], loss:[3.371/3.371], time:692162.583, lr:0.09961
Epoch time:692166.163, per step time:276.645, avg loss:3.371
...
```

### 模型验证

模型验证过程与训练过程相似。不同的是验证过程不需要设置优化器，但是需要设置评价指标

调用ImageNet验证集数据的只需要将接口的split参数设置为"val"即可，具体代码如下所示。


In [ ]:
def mobilenet_v2_eval(args_opt):
    """MobileNetV2 eval."""
    context.set_context(mode=context.GRAPH_MODE, device_target=args_opt.device_target)

    # Data pipeline.
    dataset_path = args_opt.data_url

    dataset = ImageNet(dataset_path,
                       split="val",
                       num_parallel_workers=args_opt.num_parallel_workers,
                       resize=args_opt.resize,
                       batch_size=args_opt.batch_size)

    dataset_eval = dataset.run()

    # Create model.
    network = mobilenet_v2(args_opt.num_classes, alpha=args_opt.alpha, pretrained=args_opt.pretrained,
                           resize=args_opt.resize)

    # Define loss function.
    network_loss = CrossEntropySmooth(sparse=True, reduction="mean", smooth_factor=args_opt.smooth_factor,
                                      classes_num=args_opt.num_classes)

    # Define eval metrics.
    eval_metrics = {'Top_1_Accuracy': nn.Top1CategoricalAccuracy(),
                    'Top_5_Accuracy': nn.Top5CategoricalAccuracy()}

    # Init the model.
    model = Model(network, network_loss, metrics=eval_metrics)

    # Begin to eval
    result = model.eval(dataset_eval)
    print(result)


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='MobileNetV2 eval.')
    parser.add_argument('--device_target', type=str, default="GPU", choices=["Ascend", "GPU", "CPU"])
    parser.add_argument('--data_url', required=True, default=None, help='Location of data.')
    parser.add_argument('--pretrained', type=bool, default=False, help='Load pretrained model.')
    parser.add_argument('--num_parallel_workers', type=int, default=8, help='Number of parallel workers.')
    parser.add_argument('--batch_size', type=int, default=64, help='Number of batch size.')
    parser.add_argument('--num_classes', type=int, default=1001, help='Number of classification.')
    parser.add_argument('--smooth_factor', type=float, default=0.1, help='The smooth factor.')
    parser.add_argument('--alpha', type=float, default=1.0, help='Magnification factor.')
    parser.add_argument('--resize', type=int, default=224, help='Resize the height and weight of picture.')

    args = parser.parse_known_args()[0]
    mobilenet_v2_eval(args)

运行如下脚本，即可开始验证MobileNetV2模型的精度。

```shell
python mobilenet_v2_imagenet_eval.py --alpha 0.75 --resize 192 --pretrained True --data_url ./dataset/imagenet
```

```text
{'Top_1_Accuracy': 0.6922876602564103, 'Top_5_Accuracy': 0.8871594551282052}
```

使用MindSpore Vision套件的MobileNetV2的Top-1 Accuracy和Top-5 Accuracy与使用TensorFlow的对比，如下图所示：

<div align=center><img src="./images/mobilenetv2_accuracy.png"></div>

### 模型推理

模型的推理过程较为简单，只需要使用ImageNet数据集接口读取要推理的图片，加载预训练网络，通过Model.predict方法对图片进行推理即可，具体代码如下所示。

In [ ]:
import numpy as np

from mindspore import Tensor

from mindvision.classification.utils.image import show_result
from mindvision.dataset.download import read_dataset


def mobilenet_v2_infer(args_opt):
    """MobileNetV2 infer."""
    context.set_context(mode=context.GRAPH_MODE, device_target=args_opt.device_target)

    # Data pipeline.
    dataset = ImageNet(args_opt.data_url,
                       split="infer",
                       num_parallel_workers=args_opt.num_parallel_workers,
                       resize=args_opt.resize,
                       batch_size=args_opt.batch_size)

    dataset_infer = dataset.run()

    # Create model.
    network = mobilenet_v2(args_opt.num_classes, alpha=args_opt.alpha, pretrained=args_opt.pretrained,
                           resize=args_opt.resize)

    # Init the model.
    model = Model(network)

    # Begin to infer
    image_list, _ = read_dataset(args_opt.data_url)
    for data in dataset_infer.create_dict_iterator(output_numpy=True):
        image = data["image"]
        image = Tensor(image)
        prob = model.predict(image)
        label = np.argmax(prob.asnumpy(), axis=1)
        for i, v in enumerate(label):
            predict = dataset.index2label[v]
            output = {v: predict}
            print(output)
            show_result(img=image_list[i], result=output, out_file=image_list[i])


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='MobileNetV2 infer.')
    parser.add_argument('--device_target', type=str, default="GPU", choices=["Ascend", "GPU", "CPU"])
    parser.add_argument('--data_url', required=True, default=None,
                        help='Root of infer data and ILSVRC2012_devkit_t12.tar.gz.')
    parser.add_argument('--pretrained', type=bool, default=False, help='Load pretrained model.')
    parser.add_argument('--num_parallel_workers', type=int, default=8, help='Number of parallel workers.')
    parser.add_argument('--batch_size', type=int, default=1, help='Number of batch size.')
    parser.add_argument('--num_classes', type=int, default=1001, help='Number of classification.')
    parser.add_argument('--alpha', type=float, default=1.0, help='Magnification factor.')
    parser.add_argument('--resize', type=int, default=224, help='Resize the height and weight of picture.')

    args = parser.parse_known_args()[0]
    mobilenet_v2_infer(args)

运行如下脚本，即可开始使用MobileNetV2模型对目标图片进行推理。

```shell
python mobilenet_v2_imagenet_infer.py --alpha 1.0 --resize 224 --pretrained True --data_url ./infer
```

```text
{283: 'Persian cat'}
```

<div align=center><img src="./images/mobilenetv2_infer.jpg"></div>

## 总结

本案例对MobileNetV2的论文中提出的具有线性瓶颈的倒置残差结构（Inverted Residual Block）进行了详细的解释，向读者完整地呈现了该结构提出的背景以及理论依据。

同时，通过MindSpore Vision套件，剖析了MobileNetV2的主要模块和主体结构，还完成了MobileNetV2模型在ImageNet数据上的训练，验证和推理的过程，如需完整的源码可以参考[MindSpore Vision套件](https://gitee.com/mindspore/vision/tree/master/examples/classification/mobilenetv2)。

## 引用

[1] Sandler M, Howard A, Zhu M, et al. Mobilenetv2: Inverted residuals and linear bottlenecks[C]//Proceedings of the IEEE conference on computer vision and pattern recognition. 2018: 4510-4520.